## 2020.11.06

# XSLT partial code generator for transforming Infopad form xml
## The Conditions are listed below:

|      |  Condition  | Group | Question | Answer |
|:----:|:-----------:|:-----:|:--------:|:------:|
| n.a. | Condition 0 |   0   |     0    |    0   |
|      | Condition 1 |   0   |     0    |    1   |
|      | Condition 2 |   0   |     1    |    0   |
|      | Condition 3 |   0   |     1    |    1   |
|      | Condition 4 |   1   |     0    |    0   |
| n.a. | Condition 5 |   1   |     0    |    1   |
|      | Condition 6 |   1   |     1    |    0   |
|      | Condition 7 |   1   |     1    |    1   |

In [1]:
# This makes the plots more crisp
# To enable the autocomplete feature based upon clicking the TAB button
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [2]:
import pandas as pd
import numpy as np
from decimal import *
import matplotlib.pyplot as plt

In [3]:
import datetime as datetime
sys_date = datetime.datetime.today().strftime( '%Y%m%d')
sys_date = f'{sys_date}' 
sys_date

'20201106'

In [4]:
raw = pd.read_excel('xslt_scripts/infopad-ism-variable-mapping_AJ_20201105_v1.xlsx')

In [5]:
df_raw = raw.copy(deep=True)
df_raw.head()   # size 922 x 8

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded
0,2.0,249961490.0,NaN,NaN,NaN,Klinikk,NaN,physio_id,0.0,0.0,1.0,0.0,0.0,1,NaN
1,3.0,249961490.0,NaN,NaN,NaN,Pasient nr,NaN,journal_id,0.0,0.0,1.0,0.0,0.0,1,NaN
2,4.0,249961490.0,NaN,NaN,NaN,Fødselsår,NaN,birth_year,0.0,0.0,1.0,0.0,0.0,1,NaN
3,NaN,249961490.0,SupportPRIM - Baseline Pasient,"Samtykke - SupportPRIM"" Project",FORESPØRSEL OM DELTAKELSE I FORSKNINGSPROSJEKTET,JEG SAMTYKKER TIL Å DELTA I PROSJEKTET OG TIL ...,NaN,consent_1,1.0,NaN,NaN,0.0,0.0,1,NaN
4,5.0,249961490.0,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Hvordan vil du beskrive arbeidet ditt (lønnet ...,NaN,work_type_1,1.0,NaN,NaN,0.0,0.0,1,NaN


# 
# 
## Function to write the formatted xslt code to a file

In [6]:
def save_xslt_to_file(xslt_code, file_name='xslt_code'):
    '''
    Function to write formatted xslt codes to a file.
    '''
    
    with open(file_name+'.xslt', 'w') as f:
        print(xslt_code, file=f)

## 

In [7]:
df = df_raw.dropna( subset=['infopad_group_txt', 'infopad_question_txt', 'infopad_answer_txt', 'ism_variable'], how='all')
print(df.shape)
df.head()  
# shape (481, 14)

(478, 15)


,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded
0,2.0,249961490.0,NaN,NaN,NaN,Klinikk,NaN,physio_id,0.0,0.0,1.0,0.0,0.0,1,NaN
1,3.0,249961490.0,NaN,NaN,NaN,Pasient nr,NaN,journal_id,0.0,0.0,1.0,0.0,0.0,1,NaN
2,4.0,249961490.0,NaN,NaN,NaN,Fødselsår,NaN,birth_year,0.0,0.0,1.0,0.0,0.0,1,NaN
3,NaN,249961490.0,SupportPRIM - Baseline Pasient,"Samtykke - SupportPRIM"" Project",FORESPØRSEL OM DELTAKELSE I FORSKNINGSPROSJEKTET,JEG SAMTYKKER TIL Å DELTA I PROSJEKTET OG TIL ...,NaN,consent_1,1.0,NaN,NaN,0.0,0.0,1,NaN
4,5.0,249961490.0,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Hvordan vil du beskrive arbeidet ditt (lønnet ...,NaN,work_type_1,1.0,NaN,NaN,0.0,0.0,1,NaN


In [8]:
def debug_print(codebook_id='', infopad_task_id='', infopad_task_txt='', infopad_form_txt='', 
                infopad_group_txt='', infopad_question_txt='', infopad_answer_txt='', ism_variable='', 
                normal_var='', score_var='', unique_var='', multi_var='', special_var='', valid_var=''):
    print(
            codebook_id,          '   :  \t', 
            infopad_task_id,      ' \t:  \t', 
            infopad_task_txt,     ' \t:  \t', 
            infopad_form_txt,     ' \t:  \t',
            infopad_group_txt,    ' \t:  \t', 
            infopad_question_txt, ' \t:  \t', 
            infopad_answer_txt,  ' \t:  \t', 
            ism_variable,         ' \t:  \t', 
            normal_var,               ' \t:  \t', 
            score_var,                ' \t:  \t', 
            unique_var,               ' \t:  \t', 
            multi_var,             ' \t:  \t', 
            special_var,              ' \t:  \t', 
            valid_var                   
    ) 

In [9]:
def condition_df_appender(
    df,
    codebook_id,         
    infopad_task_id,     
    infopad_task_txt,    
    infopad_form_txt,    
    infopad_group_txt,   
    infopad_question_txt,
    infopad_answer_txt, 
    ism_variable,        
    normal_var
    #,              
    #score_var,               
    #unique_var,              
    #multi_var,            
    #special_var,             
    #valid_var                
    ):
    
    df = df.append(
                    {
                        'codebook_id'           : codebook_id,   
                        'infopad_task_id'       : str(int(infopad_task_id)),
                        'infopad_task_txt'      : infopad_task_txt,
                        'infopad_form_txt'      : infopad_form_txt,
                        'infopad_group_txt'     : infopad_group_txt,              
                        'infopad_question_txt'  : infopad_question_txt,              
                        'infopad_answer_txt'    : infopad_answer_txt,                          
                        'ism_variable'          : ism_variable,      
                        'normal_var'            : int(normal_var)
                        #,  
                        #'score_var'             : int(score_var),   
                        #'unique_var'            : int(unique_var),  
                        #'multi_var'             : int(multi_var),
                        #'special_var'           : int(special_var), 
                        #'valid_var'             : int(valid_var)    
                   }, 
                   ignore_index=True)
    return df

## Logic for only extracting the group combination for creating the ism_variable

In [10]:
def populate_normal_condition_df(
    condition_df_list,
    codebook_id,         
    infopad_task_id,     
    infopad_task_txt,    
    infopad_form_txt,    
    infopad_group_txt,   
    infopad_question_txt,
    infopad_answer_txt, 
    ism_variable,        
    normal_var,              
    score_var='',               
    unique_var='',              
    multi_var='',            
    special_var='',             
    valid_var=''
    ):
    
    if ( normal_var == 1) & (infopad_task_id > 0):
    
        if  ( isinstance( infopad_group_txt    , str)  ==  False ) :
            if  ( isinstance( infopad_question_txt , str)  ==  False ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Error........................ \n')
                    condition_df_list[0] = condition_df_appender( condition_df_list[0], 
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)
                    
                    
                elif ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #print('\nCondition 2: ')
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[1] = condition_df_appender( condition_df_list[1], 
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)
                    #print(condition_df_list[1])
                    
                    
            elif ( isinstance( infopad_question_txt , str)  ==  True ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Condition 3: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[2] = condition_df_appender( condition_df_list[2], 
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)
                    
                elif ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #print('\n Condition 4: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[3] = condition_df_appender( condition_df_list[3],
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)                
                
                
        elif ( isinstance( infopad_group_txt    , str)  ==  True ) :
            if  ( isinstance( infopad_question_txt , str)  ==  False ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Condition 5: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[4] = condition_df_appender( condition_df_list[4], 
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)
                    
                elif ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #gen_answer_xslt()
                    #print('\n Condition 6: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[5] = condition_df_appender( condition_df_list[5],
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)
                    
                    
            elif ( isinstance( infopad_question_txt , str)  ==  True ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Condition 7: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[6] = condition_df_appender( condition_df_list[6],
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)
                    
                if  ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #print('\n Condition 8: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[7] = condition_df_appender( condition_df_list[7],
                                                                  codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt,
                                                                  infopad_group_txt, infopad_question_txt, infopad_answer_txt, 
                                                                  ism_variable, normal_var)


In [11]:
pd.DataFrame(columns=df.columns.tolist())

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded


In [12]:
pd.DataFrame(columns=['codebook_id', 'infopad_task_id', 'infopad_task_txt', 'infopad_form_txt', 'infopad_group_txt', 'infopad_question_txt', 'infopad_answer_txt', 'ism_variable', 'normal_var', 'score_var', 'unique_var', 'multi_var', 'special_var', 'valid_var'])

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var


In [13]:
df_cond_0 = pd.DataFrame(columns=df.columns.tolist())
df_cond_0

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded


In [14]:
df.iloc[0, :]

codebook_id                       2
infopad_task_id         2.49961e+08
infopad_task_txt                NaN
infopad_form_txt                NaN
infopad_group_txt               NaN
infopad_question_txt        Klinikk
infopad_answer_txt              NaN
ism_variable              physio_id
normal_var                        0
score_var                         0
unique_var                        1
multi_var                         0
special_var                       0
valid_var                         1
coded                           NaN
Name: 0, dtype: object

In [15]:
df_cond_0 = pd.DataFrame(columns=df.columns.tolist())
df_cond_1 = pd.DataFrame(columns=df.columns.tolist())
df_cond_2 = pd.DataFrame(columns=df.columns.tolist())
df_cond_3 = pd.DataFrame(columns=df.columns.tolist())
df_cond_4 = pd.DataFrame(columns=df.columns.tolist())
df_cond_5 = pd.DataFrame(columns=df.columns.tolist())
df_cond_6 = pd.DataFrame(columns=df.columns.tolist())
df_cond_7 = pd.DataFrame(columns=df.columns.tolist())


condition_df_list = [df_cond_0, df_cond_1, df_cond_2, df_cond_3, df_cond_4, df_cond_5, df_cond_6, df_cond_7]

zip_df = zip(df.iloc[:, 0], df.iloc[:, 1], df.iloc[:, 2], df.iloc[:, 3], df.iloc[:, 4], df.iloc[:, 5], df.iloc[:, 6], df.iloc[:, 7], df.iloc[:, 8])

for codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt, infopad_group_txt, infopad_question_txt, infopad_answer_txt, ism_variable, normal_var in zip_df:
    #print(infopad_xml_text, ism_group_var, ism_question_var, ism_answer_var, ism_variable)
    
    cbr_empty_val = '_unknown_'
    #empty_val = ''
    
    infopad_empty = 'infopad_var_not_defined'
    ism_empty = 'ism_var_not_defined'
    
    #if  ( isinstance( infopad_answer_txt, str)  ==  False  ) :
    #    infopad_answer_txt = infopad_empty
        
    #if  ( isinstance( ism_variable, str)  ==  False  ) :
    #    ism_variable = ism_empty
    
    
    populate_normal_condition_df( condition_df_list, codebook_id, infopad_task_id, infopad_task_txt, infopad_form_txt, infopad_group_txt, infopad_question_txt, infopad_answer_txt, ism_variable, normal_var)

In [16]:
i = -1
for df_test in condition_df_list:
    i += 1
    print( f'Condition - {i} : ', df_test.shape)
    
# Output 
# (0, 7)
# (0, 7)
# (239, 7)
# (55, 7)
# (36, 7)
# (0, 7)
# (37, 7)
# (85, 7)

Condition - 0 :  (0, 15)
Condition - 1 :  (0, 15)
Condition - 2 :  (1, 15)
Condition - 3 :  (0, 15)
Condition - 4 :  (0, 15)
Condition - 5 :  (0, 15)
Condition - 6 :  (200, 15)
Condition - 7 :  (137, 15)


In [17]:
condition_df_list[1]

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded


In [18]:
xslt_code_pre = f"""\
<?xml version="1.0" encoding="UTF-8"?>
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:output encoding="UTF-8" indent="yes" method="xml" version="1.0"/>
        
    <xsl:template match="/">
    
        <patientDetails>
        
            <projectId><xsl:value-of select="forms/@projectnumber"/></projectId>
            
            <!--<taskId>249961490</taskId>-->
            <!--<taskName>SupportPRIM - Baseline Pasient</taskName>-->
            
            <clinicName><xsl:value-of select="forms/form/patient/@medicalOffice"/></clinicName>
            <journalId><xsl:value-of select="forms/form/patient/@journalId"/></journalId>
            <birthYear><xsl:value-of select="forms/form/patient/@birthYear"/></birthYear>
            
            <!--Get the star date and time-->
            <xsl:for-each select="//form">
                <xsl:if test=\"@Name = 'Bakgrunnsdata - supportPRIM'\">
                    <startDate>
                        <xsl:value-of select=\"substring-before(@start, ' ')\"/>
                    </startDate>
                    <startTime>
                        <xsl:value-of select=\"substring-after(@start, ' ')\"/>
                    </startTime>
                </xsl:if>
                
            <!--</xsl:for-each> -->
            <!-- <xsl:for-each select="//form/groups/group"> -->
            <!-- <xsl:for-each select="//form"> -->
""" 
print(xslt_code_pre)

<?xml version="1.0" encoding="UTF-8"?>
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:output encoding="UTF-8" indent="yes" method="xml" version="1.0"/>
        
    <xsl:template match="/">
    
        <patientDetails>
        
            <projectId><xsl:value-of select="forms/@projectnumber"/></projectId>
            
            <!--<taskId>249961490</taskId>-->
            <!--<taskName>SupportPRIM - Baseline Pasient</taskName>-->
            
            <clinicName><xsl:value-of select="forms/form/patient/@medicalOffice"/></clinicName>
            <journalId><xsl:value-of select="forms/form/patient/@journalId"/></journalId>
            <birthYear><xsl:value-of select="forms/form/patient/@birthYear"/></birthYear>
            
            <!--Get the star date and time-->
            <xsl:for-each select="//form">
                <xsl:if test="@Name = 'Bakgrunnsdata - supportPRIM'">
                    <startDate>
                        <

In [19]:
xslt_code_post = f"""
            </xsl:for-each>
        </patientDetails>
    </xsl:template>
</xsl:stylesheet>
"""
print(xslt_code_post)


            </xsl:for-each>
        </patientDetails>
    </xsl:template>
</xsl:stylesheet>



# 
# 
## Generate XSLT for the combination of None from Infopad xml
### (Condition 0)

In [20]:
condition_df_list[0]

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded


In [21]:
def gen_condition_0_xslt(df, empty_val='_unknown_'):
    xslt = f"""
            <!-- Code for condition : 0  START ....................... -->
            """

    for infopad_answer_txt, ism_variable in zip(df.infopad_answer_txt, df.ism_variable):
        #print ( infopad_answer_txt, ism_answer_var)

        xslt += f"""
                    <xsl:if test="answer[@selected='true'] and answer='{infopad_answer_txt}'">
                        <{ism_variable}><xsl:value-of select="answer[@selected='true']"/></{ism_variable}>
                    </xsl:if>
                """
        
    xslt += f"""   
            <!-- Code for condition : 0  ......................... END -->
                
            """
    return xslt

print(gen_condition_0_xslt(condition_df_list[0]))


            <!-- Code for condition : 0  START ....................... -->
               
            <!-- Code for condition : 0  ......................... END -->
                
            


In [22]:
xslt_code_c0 = gen_condition_0_xslt(condition_df_list[0])

# 
# 
## Generate XSLT for the combination of Answer from Infopad xml
### (Condition 1)

In [23]:
condition_df_list[1][0:2]

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded


In [24]:
def gen_answer_xslt(df, empty_val='_unknown_'):
    xslt = f"""
            <!-- Code for condition : 1  START ....................... -->
            """
    
    for infopad_answer_txt, ism_variable in zip(df.infopad_answer_txt, df.ism_variable):
        #print ( infopad_answer_txt, ism_variable)

        xslt += f"""
                    <xsl:if test="answer[@selected='true'] and answer='{infopad_answer_txt}'">
                        <{ism_variable}><xsl:value-of select="answer[@selected='true']"/></{ism_variable}>
                    </xsl:if>
                """
        
    xslt += f"""   
            <!-- Code for condition : 1  ......................... END -->
                
            """
    return xslt

print(gen_answer_xslt(condition_df_list[1][0:1]))


            <!-- Code for condition : 1  START ....................... -->
               
            <!-- Code for condition : 1  ......................... END -->
                
            


In [25]:
xslt_code_c1 = gen_answer_xslt(condition_df_list[1])
print(xslt_code_c1)


            <!-- Code for condition : 1  START ....................... -->
               
            <!-- Code for condition : 1  ......................... END -->
                
            


# 
# 
## Generate XSLT for the combination of Question from Infopad xml
### (Condition 2)

In [26]:
condition_df_list[2]

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded
0,539.0,249966392,NaN,NaN,NaN,Arbeidsevne - Fysioprim,NaN,work_ability_w1,1,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def gen_question_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = f"""
            <!-- Code for condition : 2  START ....................... -->
            """
    
    xslt += f"""
                <xsl:for-each select="groups/group/question">
            """
    
    for infopad_task_id, infopad_form_txt, infopad_question_txt, ism_variable in zip(df.infopad_task_id, df.infopad_form_txt, df.infopad_question_txt, df.ism_variable):
        
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
            
        xslt += f"""
                    <xsl:if test="../../../@taskid = '{infopad_task_id}'">
                        <xsl:if test="../../../@Name = '{infopad_form_txt}'">
                            <xsl:if test="@name='{infopad_question_txt}'">
                                <{ism_variable}>
                                    <xsl:choose>
                                        <xsl:when test="answer/@selected='true'">
                                            <xsl:value-of select="answer[@selected='true']"/>
                                        </xsl:when>
                                        <xsl:when test="answer[not(@selected)]">
                                            <xsl:value-of select="answer"/>
                                        </xsl:when>
                                        <xsl:otherwise>{empty_val}</xsl:otherwise>
                                    </xsl:choose>
                                </{ism_variable}>
                            </xsl:if>
                         </xsl:if>
                     </xsl:if>
                """
        
    xslt += f"""    
                </xsl:for-each>
            """
    
    xslt += f"""   
            <!-- Code for condition : 1  ......................... END -->
                
            """
    
    return xslt

print(gen_question_xslt(condition_df_list[2][0:10]))


            <!-- Code for condition : 2  START ....................... -->
            
                <xsl:for-each select="groups/group/question">
            
                    <xsl:if test="../../../@taskid = '249966392'">
                        <xsl:if test="../../../@Name = 'nan'">
                            <xsl:if test="@name='Arbeidsevne - Fysioprim'">
                                <work_ability_w1>
                                    <xsl:choose>
                                        <xsl:when test="answer/@selected='true'">
                                            <xsl:value-of select="answer[@selected='true']"/>
                                        </xsl:when>
                                        <xsl:when test="answer[not(@selected)]">
                                            <xsl:value-of select="answer"/>
                                        </xsl:when>
                                        <xsl:otherwise>_unknown_</xsl:otherwise>
             

In [28]:
xslt_code_c2 = gen_question_xslt(condition_df_list[2])
#print(xslt_code_c2)

# 
# 
## Generate XSLT for the combination of Question & Answer from Infopad xml
### (Condition 3)

In [29]:
condition_df_list[3].tail()

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded


In [30]:
infopad_empty = 'infopad_var_not_defined'
isinstance(infopad_answer_txt, str)

False

In [31]:
def gen_question_answer_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):

    xslt = f"""
            <!-- Code for condition : 3  START ....................... -->
            """
    
    xslt += f"""
                <xsl:for-each select="groups/group/question">
            """
    
    infopad_empty = 'infopad_var_not_defined'
    ism_empty = 'ism_var_not_defined'
    
    for infopad_task_id, infopad_question_txt, infopad_answer_txt, ism_variable in zip( df.infopad_task_id, df.infopad_question_txt,	df.infopad_answer_txt, df.ism_variable):
        
        #print(infopad_task_id, ' ', infopad_question_txt,  ' ', infopad_answer_txt,  ' ', ism_variable)
        
        flag = True
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
            flag = False
            #print(infopad_task_id)
            #xslt += f"""
            #            <!-- infopad_question_txt is empty -->
            #        """
        if (isinstance(infopad_answer_txt, str) & (infopad_answer_txt != infopad_empty)) == False:
            infopad_answer_txt = empty_val
            flag = False
            #print(infopad_task_id, ' ', infopad_question_txt)
            #xslt += f"""
            #            <!-- infopad_question_txt is empty -->
            #        """
        if (isinstance(ism_variable, str) & (ism_variable != ism_empty)) == False:
            ism_variable = empty_key
            flag = False
            #print(infopad_task_id, ' ', infopad_question_txt,  ' ', infopad_answer_txt,  ' ', ism_variable)
            #xslt += f"""
            #            <!-- infopad_question_txt is empty -->
            #        """
        
        
        if flag:
            #print(flag)        
            xslt += f"""
                        <xsl:if test="../../../@taskid = '{infopad_task_id}'">
                            <xsl:if test="@name = '{infopad_question_txt}'">
                                <xsl:for-each select="answer">
                                    <xsl:if test=".='{infopad_answer_txt}'">
                                        <{ism_variable}><xsl:value-of select="./@selected"/></{ism_variable}>
                                    </xsl:if>
                                </xsl:for-each>
                            </xsl:if>
                        </xsl:if>
                    """
                
    
    xslt += f"""
                </xsl:for-each>
                
            <!-- Code for condition : 3  ......................... END -->
            
            
            """

    return xslt
        
print(gen_question_answer_xslt(condition_df_list[3][0:1]))


            <!-- Code for condition : 3  START ....................... -->
            
                <xsl:for-each select="groups/group/question">
            
                </xsl:for-each>
                
            <!-- Code for condition : 3  ......................... END -->
            
            
            


In [32]:
xslt_code_c3 = gen_question_answer_xslt(condition_df_list[3])
#print(xslt_code_c3)

# 
# 
## Generate XSLT for the combination of Group from Infopad xml
### (Condition 4)

In [33]:
def gen_group_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = f"""   
            <!-- Code for condition : 4  START ....................... -->
            """
    
    for infopad_task_id, infopad_group_txt, ism_variable in zip( df.infopad_task_id, df.infopad_group_txt, df.ism_variable):
        if isinstance(infopad_group_txt, str) == False:
            infopad_group_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key

        xslt += f"""
                <xsl:if test="@taskid = '{infopad_task_id}'">
                    <xsl:for-each select="groups/group">
                        <xsl:if test="@name='{infopad_group_txt}'">
                            <xsl:for-each select="question">
                                <xsl:choose>
                                    <xsl:when test="answer/@selected='true'">
                                        <{ism_variable}><xsl:value-of select="answer"/></{ism_variable}>
                                    </xsl:when>
                                    <xsl:when test="answer[not(@selected)]">
                                        <{ism_variable}><xsl:value-of select="answer"/></{ism_variable}>
                                    </xsl:when>
                                </xsl:choose>
                            </xsl:for-each>
                        </xsl:if>
                    </xsl:for-each>
                </xsl:if>
                """
        
    xslt += f"""
            <!-- Code for condition : 4  ......................... END -->
            
            """
    
    return xslt
        
print(gen_group_xslt(condition_df_list[4][0:1]))

   
            <!-- Code for condition : 4  START ....................... -->
            
            <!-- Code for condition : 4  ......................... END -->
            
            


In [34]:
xslt_code_c4 = gen_group_xslt(condition_df_list[4])
#print(xslt_code_c4)

# 
# 
## Generate XSLT for the combination of Group and Question from Infopad xml
### (Condition 6)

In [35]:
xslt_code_c5 = ""

In [36]:
condition_df_list[6].head()

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded
0,NaN,249961490,SupportPRIM - Baseline Pasient,"Samtykke - SupportPRIM"" Project",FORESPØRSEL OM DELTAKELSE I FORSKNINGSPROSJEKTET,JEG SAMTYKKER TIL Å DELTA I PROSJEKTET OG TIL ...,NaN,consent_1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Hvordan vil du beskrive arbeidet ditt (lønnet ...,NaN,work_type_1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Arbeidsevne,NaN,work_ability_1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,"Når du tenker på barndommen/ oppveksten din, v...",NaN,childhood_1,1,NaN,NaN,NaN,NaN,NaN,NaN
4,30.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Hvor redusert er ditt daglige aktivitetsnivå p...,NaN,activity_1,1,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
def gen_group_question_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = f"""   
            <!-- Code for condition : 6  START ....................... -->
            """
    
    for infopad_task_id, infopad_form_txt, infopad_group_txt, infopad_question_txt, ism_variable in zip(df.infopad_task_id, df.infopad_form_txt, df.infopad_group_txt, df.infopad_question_txt, df.ism_variable):
        
        if isinstance(infopad_group_txt, str) == False:
            infopad_group_txt = empty_val
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
            
        xslt += f"""     
                    <xsl:if test="@taskid = '{infopad_task_id}'">
                        <xsl:if test="@Name='{infopad_form_txt}'">
                            <xsl:for-each select="groups/group">
                                <xsl:if test="@name='{infopad_group_txt}'">
                                    <xsl:for-each select="question">
                                        <xsl:if test="@name = '{infopad_question_txt}'">
                                            <{ism_variable}>
                                                <xsl:choose>
                                                    <xsl:when test="answer/@selected = 'true'">
                                                        <xsl:value-of select="answer[@selected = 'true']"/>
                                                    </xsl:when>
                                                    <xsl:when test="answer[not(@selected)]">
                                                        <xsl:value-of select="answer"/>
                                                    </xsl:when>
                                                    <xsl:otherwise>{empty_val}</xsl:otherwise>
                                                </xsl:choose>
                                            </{ism_variable}>
                                        </xsl:if>
                                    </xsl:for-each>
                                </xsl:if>
                            </xsl:for-each>
                        </xsl:if>
                    </xsl:if>
                """
        
    xslt += f"""
            <!-- Code for condition : 6  ......................... END -->
            
            """
    
    return xslt
                          
print(gen_group_question_xslt(condition_df_list[6][0:2]))

   
            <!-- Code for condition : 6  START ....................... -->
                 
                    <xsl:if test="@taskid = '249961490'">
                        <xsl:if test="@Name='Samtykke - SupportPRIM" Project'">
                            <xsl:for-each select="groups/group">
                                <xsl:if test="@name='FORESPØRSEL OM DELTAKELSE I FORSKNINGSPROSJEKTET'">
                                    <xsl:for-each select="question">
                                        <xsl:if test="@name = 'JEG SAMTYKKER TIL Å DELTA I PROSJEKTET OG TIL AT MINE PERSONOPPLYSNINGER BRUKES SLIK DET ER BESKREVET'">
                                            <consent_1>
                                                <xsl:choose>
                                                    <xsl:when test="answer/@selected = 'true'">
                                                        <xsl:value-of select="answer[@selected = 'true']"/>
                                     

In [70]:
xslt_code_c6 = gen_group_question_xslt(condition_df_list[6])
#print(xslt_code_c6)

# 
# 
## Generate XSLT for the combination of Group, Question, and Answer from Infopad xml
### (Condition 7)

In [71]:
condition_df_list[7].head()

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,normal_var,score_var,unique_var,multi_var,special_var,valid_var,coded
0,16.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Andre nåværende sykdommer eller plager:,Revmatisk sykdom,como_ra_1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,17.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Andre nåværende sykdommer eller plager:,Hjertesykdom,como_heart_1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,18.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Andre nåværende sykdommer eller plager:,Astma/ lungesykdom,como_asthma_1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,19.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Andre nåværende sykdommer eller plager:,Nevrologisk sykdom (f.eks tidligere hjerneslag...,como_neuro_1,1,NaN,NaN,NaN,NaN,NaN,NaN
4,20.0,249961490,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Andre nåværende sykdommer eller plager:,Kreft/ tidligere kreft,como_cancer_1,1,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
def gen_group_question_answer_xslt(df, empty_val='_unknown_'):
    xslt = f"""
            <!-- Code for condition : 7  START ....................... -->
            """

    for infopad_task_id, infopad_form_txt, infopad_group_txt, infopad_question_txt, infopad_answer_txt, ism_variable in zip( 
        df.infopad_task_id, df.infopad_form_txt, df.infopad_group_txt, df.infopad_question_txt,	df.infopad_answer_txt, df.ism_variable):
        
        if isinstance(infopad_form_txt, str) == False:
            infopad_form_txt = empty_val
        if isinstance(infopad_group_txt, str) == False:
            infopad_group_txt = empty_val
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(infopad_answer_txt, str) == False:
            infopad_answer_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
            
            #<xsl:for-each select="//form">
            #    <xsl:if test="@Name = 'Ryggundersøkelse - SupportPRIM'">
            #        <xsl:for-each select="groups/group">
            #            <xsl:if test="@name = 'RYGGUNDERSØKELSE'">
            #                <xsl:for-each select="question">
            #                    <xsl:if test="@name = 'Reduserte bevegelsesutslag i'">
            #                        <xsl:for-each select="answer">
    
        xslt += f"""
                <xsl:if test="@taskid = '{infopad_task_id}'">
                    <xsl:if test="@Name = '{infopad_form_txt}'">
                        <xsl:for-each select="groups/group">
                            <xsl:if test="@name = '{infopad_group_txt}'">
                                <xsl:for-each select="question">
                                    <xsl:if test="@name = '{infopad_question_txt}'">
                                        <xsl:for-each select="answer">
                                            <xsl:if test=".='{infopad_answer_txt}'">
                                                <{ism_variable}>
                                                    <xsl:value-of select="./@selected"/>
                                                </{ism_variable}>
                                            </xsl:if>
                                        </xsl:for-each>
                                    </xsl:if>
                                </xsl:for-each>
                            </xsl:if>
                        </xsl:for-each>
                    </xsl:if>
                </xsl:if>
        """
        
    xslt += f"""
            <!-- Code for condition : 7  ......................... END -->
            
            
            """
    return xslt

print(gen_group_question_answer_xslt(condition_df_list[7][0:1]))


            <!-- Code for condition : 7  START ....................... -->
            
                <xsl:if test="@taskid = '249961490'">
                    <xsl:if test="@Name = 'Bakgrunnsdata - supportPRIM'">
                        <xsl:for-each select="groups/group">
                            <xsl:if test="@name = 'Bakgrunnsdata '">
                                <xsl:for-each select="question">
                                    <xsl:if test="@name = 'Andre nåværende sykdommer eller plager:'">
                                        <xsl:for-each select="answer">
                                            <xsl:if test=".='Revmatisk sykdom'">
                                                <como_ra_1>
                                                    <xsl:value-of select="./@selected"/>
                                                </como_ra_1>
                                            </xsl:if>
                                        </xsl:for-each>
              

In [73]:
xslt_code_c7 = gen_group_question_answer_xslt(condition_df_list[7])

In [74]:
#print(xslt_code_c7)

# 
# 

## Code for PSFS

In [75]:
df.ism_variable

0                       physio_id
1                      journal_id
2                      birth_year
3                       consent_1
4                     work_type_1
5                  work_ability_1
6                     childhood_1
7                          como_1
8                       como_ra_1
9                    como_heart_1
10                  como_asthma_1
11                   como_neuro_1
12                  como_cancer_1
13                   como_psych_1
14               como_dementhia_1
15            como_osteoporosis_1
16                como_diabetes_1
17            como_stomach_pain_1
18                como_headache_1
19                 como_fatigue_1
20                   como_other_1
21              como_other_spec_1
22                     activity_1
23                         fear_1
24                     treat12m_1
25                treat12m_none_1
26                  treat12m_gp_1
27             treat12m_special_1
28               treat12m_chiro_1
29            

In [76]:
df_psfs_act = df.loc[df['ism_variable'].str.startswith('psfs1_act', na=False)]

In [77]:
for i, row in df_psfs_act.iterrows():
    print(int(row.infopad_task_id), row.infopad_form_txt, row.infopad_group_txt, row.ism_variable)

249961492 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_1
249966392 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_w1
249966394 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_w2
249966420 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_w4
249966422 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_w8
249966782 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_p_3
249966786 Pasientspesifikk funksjonsskala - SupportPRIM Pasientspesifikk funksjonsskala psfs1_act_3


In [78]:
task_id = 249961492
form_name = 'Pasientspesifikk funksjonsskala - SupportPRIM'
act_var = 'psfs1_act_1'
psfs_val = act_var.replace('psfs1_act', 'psfs1')


In [79]:
def gen_psfs_xslt(df, empty_val='_unknown_'):
    xslt = f"""
            <!-- Code for PSFS  START ....................... -->
            """
    
    df_psfs_act = df.loc[df['ism_variable'].str.startswith('psfs1_act', na=False)]
    
    for i, row in df_psfs_act.iterrows():
        
        #print(int(row.infopad_task_id), row.infopad_form_txt, row.infopad_group_txt, row.ism_variable)
        
        task_id = int(row.infopad_task_id)
        form_name = row.infopad_form_txt
        group_name = row.infopad_group_txt
        psfs_act_var = row.ism_variable
        psfs_val_var = act_var.replace('psfs1_act', 'psfs1')
    
        xslt += f"""
                    <xsl:if test="@taskid = '{task_id}'">
                        <xsl:for-each select="groups/group">
                            <xsl:if test="@name='{group_name}'">
                                <{psfs_act_var}>
                                    <xsl:value-of select="question/@name"/>
                                </{psfs_act_var}> 
                                <xsl:for-each select="question[1]/answer">
                                    <xsl:choose>
                                        <xsl:when test="@selected='true'">
                                            <{psfs_val_var}><xsl:value-of select="."/></{psfs_val_var}>
                                        </xsl:when>
                                    </xsl:choose> 
                                </xsl:for-each>                  
                            </xsl:if>
                        </xsl:for-each>
                    </xsl:if>
                """
        
    xslt += f"""
            <!-- Code for PSFS  ......................... END -->
            
            
            """
    return xslt

#print(gen_psfs_xslt(df))

In [80]:
xslt_code_psfs = gen_psfs_xslt(df)
#print(xslt_code_psfs)

## Code for painsites

In [81]:
df.ism_variable[0]

'physio_id'

In [82]:
df_painsite = df.loc[df['ism_variable'].str.startswith('painsite_', na=False)]

In [83]:
df_painsite;

In [84]:
df_painsite.infopad_form_txt.iloc[0]

'Smertetegning siste uke (med regional angivelse)'

In [85]:
for i, row in df_painsite.iterrows():
    print(int(row.infopad_task_id), row.infopad_form_txt, row.infopad_group_txt, row.infopad_question_txt, row.ism_variable)

249961490 Smertetegning siste uke (med regional angivelse) nan Hode painsite_head_1
249961490 Smertetegning siste uke (med regional angivelse) nan Nakke painsite_neck_1
249961490 Smertetegning siste uke (med regional angivelse) nan Skuldre painsite_shou_1
249961490 Smertetegning siste uke (med regional angivelse) nan Øvre del rygg painsite_uback_1
249961490 Smertetegning siste uke (med regional angivelse) nan Albuer painsite_elbow_1
249961490 Smertetegning siste uke (med regional angivelse) nan Håndledd / hender painsite_hand_1
249961490 Smertetegning siste uke (med regional angivelse) nan Hofter / lår painsite_thigh_1
249961490 Smertetegning siste uke (med regional angivelse) nan Korsrygg painsite_lback_1
249961490 Smertetegning siste uke (med regional angivelse) nan Ankler / føtter painsite_feet_1
249961490 Smertetegning siste uke (med regional angivelse) nan Knær painsite_knee_1
249961490 Smertetegning siste uke (med regional angivelse) nan Totalt antall ruter: painsite_total_sq_1
2

In [86]:
def gen_painsite_xslt(df, empty_val='_unknown_'):
    
    df_painsite = df.loc[df['ism_variable'].str.startswith('painsite_', na=False)]
    
    task_id = int(df_painsite.infopad_task_id.iloc[0])
    form_name = df_painsite.infopad_form_txt.iloc[0]
    
    xslt = f"""
                <!-- Code for Painsites  START ....................... -->
                    <xsl:if test="@taskid = '{task_id}'">
                        <xsl:if test="@Name='{form_name}'">
                            <xsl:for-each select="scores/score">
                                    <xsl:choose>            
            """
    
    for i, row in df_painsite.iterrows():
        
        #print(int(row.infopad_task_id), row.infopad_form_txt, row.infopad_group_txt, row.ism_variable)
        
        #task_id = int(row.infopad_task_id)
        #form_name = row.infopad_form_txt
        score_name = row.infopad_question_txt
        painsite_var = row.ism_variable
    
        xslt += f"""
                                        <xsl:when test=". = '{score_name}'">
                                            <{painsite_var}>
                                                <xsl:value-of select="@points"/>
                                            </{painsite_var}>
                                        </xsl:when>
                """
        
    xslt += f"""
                                    </xsl:choose>
                            </xsl:for-each>
                        </xsl:if>
                    </xsl:if>
                <!-- Code for Painsites  ......................... END -->
            
            
            """
    return xslt

#print(gen_psfs_xslt(df))

In [87]:
xslt_code_painsite = gen_painsite_xslt(df)
#print(xslt_code_painsite)

## Code for body_map

In [88]:
xslt_code_body_map = f"""

            <!-- Start: Code for BodyMaps .................... -->

                <xsl:if test="@taskid = '249961490'">
                    <xsl:if test="@Name='Smertetegning siste uke (med regional angivelse)'">
                        <body_map>
                            <back>
                                <xsl:for-each select="groups/group/question/bodymap/square">
                                    <xsl:if test="@side =1">
                                        <b>
                                            <x><xsl:value-of select="@x"/></x>
                                            <y><xsl:value-of select="@y"/></y>
                                        </b>
                                    </xsl:if>
                                </xsl:for-each>
                            </back>

                            <front>
                                <xsl:for-each select="groups/group/question/bodymap/square">
                                    <xsl:if test="@side =0">
                                        <f>
                                            <x><xsl:value-of select="@x"/></x>
                                            <y><xsl:value-of select="@y"/></y>
                                        </f>
                                    </xsl:if>
                                </xsl:for-each>
                            </front> 
                        </body_map>
                    </xsl:if>
                </xsl:if>

            <!-- Code for BodyMaps  ......................... END -->

"""

In [89]:
print(xslt_code_body_map)



            <!-- Start: Code for BodyMaps .................... -->

                <xsl:if test="@taskid = '249961490'">
                    <xsl:if test="@Name='Smertetegning siste uke (med regional angivelse)'">
                        <body_map>
                            <back>
                                <xsl:for-each select="groups/group/question/bodymap/square">
                                    <xsl:if test="@side =1">
                                        <b>
                                            <x><xsl:value-of select="@x"/></x>
                                            <y><xsl:value-of select="@y"/></y>
                                        </b>
                                    </xsl:if>
                                </xsl:for-each>
                            </back>

                            <front>
                                <xsl:for-each select="groups/group/question/bodymap/square">
                                    <xsl:if test="@s

# 
# 

## Code for Treatment

In [90]:
df.ism_variable[0]

'physio_id'

In [91]:
df_treat = df.loc[df['ism_variable'].str.startswith('treat_', na=False)]

In [92]:
for i, row in df_treat.iterrows():
    print(int(row.infopad_task_id), row.infopad_form_txt, row.infopad_group_txt, row.infopad_question_txt, row.ism_variable)

249961490 Bakgrunnsdata - supportPRIM Tro på behandlingen Jeg tror at fysioterapi vil bedre min funksjon: treat_belief_1
249961492 Hovedmål - supportPRIM Hovedmål med behandlingen Hva er hovedmålet med behandlingen treat_goal_1
249966782 Fysioprim - behandlingsevaluering Behandlingsevaluering I hvilken grad har du hatt nytte av fysioterapibehandling? treat_benefit_3
249966782 Fysioprim - behandlingsevaluering Behandlingsevaluering I hvilken grad ble dine forventninger til fysioterapi innfridd? treat_expect_3
249966786 Fortsetter behandlingen og antall konsultasjoner - supportPRIM Fortsetter beandlingen og antall konsultasjoner Fortsetter behandlingen? treat_contin_3
249966786 Tiltaksliste - supportPRIM 1. Redusert aktivitet, hvile, avlastning nan treat_advice_rest_3
249966786 Tiltaksliste - supportPRIM 2. Økt fysisk aktivitet nan treat_advice_pa_3
249966786 Tiltaksliste - supportPRIM 3. Å Motvirke frykt for bevegelse og aktivitet nan treat_advice_fear_3
249966786 Tiltaksliste - support

In [93]:
def gen_treatment_xslt(df, empty_val='_unknown_'):
    xslt = f"""
            <!-- Code for Treatment  START ....................... -->
            """
    
    df_treat = df.loc[df['ism_variable'].str.startswith('treat_', na=False) & df['special_var'] ==1]
    
    for i, row in df_treat.iterrows():
        
        #print(int(row.infopad_task_id), row.infopad_form_txt, row.infopad_group_txt, row.ism_variable)
        
        task_id = int(row.infopad_task_id)
        form_name = row.infopad_form_txt
        group_name = row.infopad_group_txt
        treat_var = row.ism_variable
    
        xslt += f"""
                    <xsl:if test="@taskid = '{task_id}'">
                        <xsl:if test="@Name='{form_name}'">
                            <xsl:for-each select="groups/group">
                                <xsl:if test="@name='{group_name}'">
                                    <xsl:for-each select="question/answer">
                                        <xsl:if test="@selected = 'true'">
                                            <{treat_var}>
                                                <xsl:value-of select="."/>
                                            </{treat_var}>
                                        </xsl:if>
                                    </xsl:for-each>
                                 </xsl:if>
                            </xsl:for-each>
                        </xsl:if>
                    </xsl:if>
                """
        
    xslt += f"""
            <!-- Code for Treatment  ......................... END -->
            
            
            """
    return xslt

#print(gen_psfs_xslt(df))

In [94]:
xslt_code_treat = gen_treatment_xslt(df)
#print(xslt_code_treat)

In [95]:
xslt_code_treat_describe = f"""
                <!-- Code for Treatment  START ....................... -->
            
                    <xsl:if test="@taskid = '249966786'">
                        <xsl:if test="@Name='Beskrivelse av tiltak terapeut - supportPRIM'">
                            <xsl:for-each select="groups/group">
                                <xsl:if test="@name='Beskriv tiltak som pasienten har fått så godt at en fysioterapikollega kan gjenta behandlingen mest mulig likt deg på sin pasient.'">
                                    <xsl:if test="question/@name='Viktig bakgrunnsinformasjon for behandlingsutfall'">
                                        <treat_describe_important_3>
                                            <xsl:value-of select="question/answer"/>
                                        </treat_describe_important_3>
                                    </xsl:if>                                
                                    <xsl:if test="question/@name='Pasienten har fått følgende behandling:'">
                                        <treat_describe_3>
                                            <xsl:value-of select="question/answer"/>
                                        </treat_describe_3>
                                    </xsl:if>
                                </xsl:if>
                            </xsl:for-each>
                        </xsl:if>
                    </xsl:if>
                
            <!-- Code for Treatment Describe ......................... END -->
            
            """

In [96]:
print(xslt_code_treat_describe)


                <!-- Code for Treatment  START ....................... -->
            
                    <xsl:if test="@taskid = '249966786'">
                        <xsl:if test="@Name='Beskrivelse av tiltak terapeut - supportPRIM'">
                            <xsl:for-each select="groups/group">
                                <xsl:if test="@name='Beskriv tiltak som pasienten har fått så godt at en fysioterapikollega kan gjenta behandlingen mest mulig likt deg på sin pasient.'">
                                    <xsl:if test="question/@name='Viktig bakgrunnsinformasjon for behandlingsutfall'">
                                        <treat_describe_important_3>
                                            <xsl:value-of select="question/answer"/>
                                        </treat_describe_important_3>
                                    </xsl:if>                                
                                    <xsl:if test="question/@name='Pasienten har fått fø

# 
# ..........................................................................................................................................................
# 

# Finally write XSLT code to a file

In [97]:
final_xslt = xslt_code_pre
final_xslt += xslt_code_c0 + xslt_code_c1 + xslt_code_c2 + xslt_code_c3 + xslt_code_c4 + xslt_code_c5 + xslt_code_c6 + xslt_code_c7 
final_xslt += xslt_code_painsite + xslt_code_psfs + xslt_code_treat + xslt_code_treat_describe + xslt_code_body_map
final_xslt += xslt_code_post

In [98]:
save_xslt_to_file( xslt_code=final_xslt, file_name=f'xslt_scripts/xslt_code_for_infopad_{sys_date}')
save_xslt_to_file( xslt_code=final_xslt, file_name='xslt_code_for_infopad')